In [137]:
import itertools as it
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers

Possible strategies:
        -- Need to be observed to counter:

- fixed sequence of 'n' moves
        -- previous 'n'+1 moves

- (opponent's / self's) last 'n'[0-10] moves
        -- previous n moves

- (opponent's / self's) most common move in the last 'n'[2-1000] matches
        -- ...

- (opponent's / self's) most common 'm-moves'[2-10] sequence in the last 'n'[2-1000] matches
        -- ...

In [138]:
#variables:
num = 10 #number of previous matches to analyze
seq_size = range(2, 6) #sequence sizes to test
matches_range = [5, 10] #number of previous matches to look for moves
length = max(num, max(seq_size), max(matches_range)) + 1 #length of timestep to analize. +1 for the move to predict

In [139]:
## Some functions that will be used in the main script ##

#function for counting the number of occurences of a substring in a string
def count_substring(s, sub):
    # Initialize count to zero
    cnt = 0

    # Iterate through the string to check for the substring
    for i in range(len(s) - len(sub) + 1):
        if s[i:i + len(sub)] == sub:  # If substring matches, increment count
            cnt += 1
    return cnt

In [140]:
opponent_hist = "RRRPPPSSS" * 10
player_hist = "RPS" * 30
player_hist

'RPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSRPS'

In [ ]:
vocab_moves = sorted(['R', 'P', 'S'])
# Creating a mapping from unique characters to indices
'''char2idx = {u:i for i, u in enumerate(vocab_moves)}
idx2char = np.array(vocab_moves)

def text_to_int(text):
  return [char2idx[c] for c in text]

def int_to_text(ints):
  return ''.join(idx2char[ints])

idx2char'''

array(['P', 'R', 'S'], dtype='<U1')

In [142]:
#vocab of sequences with 2-5 elements
vocab_seq = {}
for i in seq_size:
  vocab_seq[i] = []
  for seq in it.product(vocab_moves, repeat=i):
    vocab_seq[i].append(''.join(seq))

#full vocabulary
vocab = vocab_moves.copy()
for i in seq_size:
  for seq in it.product(vocab_moves, repeat=i):
    vocab.append(''.join(seq))

def int_encoding(text):
  return vocab.index(text)

def int_decoding(ints):
  return vocab[ints]

def text_to_int(text):
  return [int_encoding(c) for c in text]

def int_to_text(ints):
  return ''.join([int_decoding(i) for i in ints])

In [ ]:
def get_features(seq, encoded=True):

    opponent_hist = seq[: len(seq)//2]
    player_hist = seq[len(seq)//2 :]
    #print('opponent_hist: ', opponent_hist)
    #print('player_hist  : ', player_hist)

    #opponent's and player's last 'i'th move in the last 'num' matches:
    op_last_move = [opponent_hist[-i - 1] for i in range(num)] #[op last move, op 2nd last move, ..., op length'th last move]
    pl_last_move = [player_hist[-i - 1] for i in range(num)] #[pl last move, pl 2nd last move, ..., pl length'th last move]
    last_move = op_last_move + pl_last_move
    #print('last_move: ', last_move)

    #opponent's and player's most common move:
    op_common_all = max(set(vocab_moves), key = lambda x: count_substring(opponent_hist, x))
    pl_common_all = max(set(vocab_moves), key = lambda x: count_substring(player_hist, x))
    common_all = [op_common_all, pl_common_all]
    #print('common_all: ', common_all)

    #opponent's and player's most common move in the last 'matches_range' moves:
    op_common = []
    pl_common = []
    for i in matches_range:
        os = opponent_hist[-i :]
        op_common.append(max(set(os), key = lambda x: count_substring(os, x)))
        ps = player_hist[-i :]
        pl_common.append(max(set(ps), key = lambda x: count_substring(ps, x)))
    common = op_common + pl_common
    #print('common: ', common)
    
    #opponent's and player's last sequences of 'seq_size' moves:
    op_last_seq = [opponent_hist[-i :] for i in seq_size]
    pl_last_seq = [player_hist[-i :] for i in seq_size]
    last_seq = op_last_seq + pl_last_seq
    #print('last_seq: ', last_seq)

    #opponent's and player's most common sequence of 'seq_size' moves in the last 'matches_range' matches
    op_common_seq = []
    pl_common_seq = []
    for i in seq_size:
        for j in matches_range:
            os = opponent_hist[-j :]
            op_common_seq.append(max(set(vocab_seq[i]), key = lambda x: count_substring(os, x)))
            ps = player_hist[-j :]
            pl_common_seq.append(max(set(vocab_seq[i]), key = lambda x: count_substring(ps, x)))
    common_seq = op_common_seq + pl_common_seq
    #print('common_seq: ', common_seq)

    #opponent's and player's most common sequence of 'seq_size' moves in all matches
    op_common_seq_all = []
    pl_common_seq_all = []
    for i in seq_size:
        op_common_seq_all.append(max(set(vocab_seq[i]), key = lambda x: count_substring(opponent_hist, x)))
        pl_common_seq_all.append(max(set(vocab_seq[i]), key = lambda x: count_substring(player_hist, x)))
    common_seq_all = op_common_seq_all + pl_common_seq_all
    #print('common_seq_all: ', common_seq_all)

    features = last_move + common_all + common + last_seq + common_seq + common_seq_all
    if encoded:
        features = [int_encoding(f) for f in features]
    return features

features_example = get_features(opponent_hist + player_hist)
print('\nFEATURES: ', features_example)


FEATURES:  [2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 1, 1, 2, 2, 0, 2, 11, 38, 65, 146, 5, 23, 104, 185, 11, 3, 14, 25, 47, 47, 146, 146, 5, 10, 33, 33, 60, 104, 185, 222, 3, 25, 47, 146, 5, 23, 73, 222]


In [ ]:
def get_features_from_timestep(timestep, encoded=True): #timestep is a list of all opponent's and player's moves
    opponent_hist = timestep[: len(timestep)//2 - 1]
    player_hist = timestep[len(timestep)//2 : -1]
    seq = opponent_hist + player_hist
    features = get_features(seq, encoded)
    return features

In [ ]:
def get_label(timestep, encoded=True):
    opponent_hist = timestep[: len(timestep)//2]
    label = opponent_hist[-1]
    if encoded:
        label = int_encoding(label)
    return label

label_example = get_label(opponent_hist + player_hist)
print('LABEL: ', label_example)

LABEL:  2


In [ ]:
def get_timesteps(hist):
    op_hist = hist[: len(hist)//2]
    pl_hist = hist[len(hist)//2 :]
    timesteps = []
    for i in range(length, len(op_hist) + 1):
        timesteps.append(op_hist[:i] + pl_hist[:i])
    return timesteps

timesteps_example = get_timesteps(opponent_hist + player_hist)
print(timesteps_example)

['RRRPPPSSSRRRPSRPSRPSRP', 'RRRPPPSSSRRRRPSRPSRPSRPS', 'RRRPPPSSSRRRPRPSRPSRPSRPSR', 'RRRPPPSSSRRRPPRPSRPSRPSRPSRP', 'RRRPPPSSSRRRPPPRPSRPSRPSRPSRPS', 'RRRPPPSSSRRRPPPSRPSRPSRPSRPSRPSR', 'RRRPPPSSSRRRPPPSSRPSRPSRPSRPSRPSRP', 'RRRPPPSSSRRRPPPSSSRPSRPSRPSRPSRPSRPS', 'RRRPPPSSSRRRPPPSSSRRPSRPSRPSRPSRPSRPSR', 'RRRPPPSSSRRRPPPSSSRRRPSRPSRPSRPSRPSRPSRP', 'RRRPPPSSSRRRPPPSSSRRRRPSRPSRPSRPSRPSRPSRPS', 'RRRPPPSSSRRRPPPSSSRRRPRPSRPSRPSRPSRPSRPSRPSR', 'RRRPPPSSSRRRPPPSSSRRRPPRPSRPSRPSRPSRPSRPSRPSRP', 'RRRPPPSSSRRRPPPSSSRRRPPPRPSRPSRPSRPSRPSRPSRPSRPS', 'RRRPPPSSSRRRPPPSSSRRRPPPSRPSRPSRPSRPSRPSRPSRPSRPSR', 'RRRPPPSSSRRRPPPSSSRRRPPPSSRPSRPSRPSRPSRPSRPSRPSRPSRP', 'RRRPPPSSSRRRPPPSSSRRRPPPSSSRPSRPSRPSRPSRPSRPSRPSRPSRPS', 'RRRPPPSSSRRRPPPSSSRRRPPPSSSRRPSRPSRPSRPSRPSRPSRPSRPSRPSR', 'RRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPSRPSRPSRPSRPSRPSRPSRPSRPSRP', 'RRRPPPSSSRRRPPPSSSRRRPPPSSSRRRRPSRPSRPSRPSRPSRPSRPSRPSRPSRPS', 'RRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPRPSRPSRPSRPSRPSRPSRPSRPSRPSRPSR', 'RRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPP

In [147]:
def get_features_list(hist, encoded = True):
    timesteps = get_timesteps(hist)
    features = []
    for timestep in timesteps:
        features.append(get_features_from_timestep(timestep, encoded))
    return np.array(features)
features_list_example = get_features_list(opponent_hist + player_hist)
features_list_example.shape

(80, 58)

In [148]:
def get_labels_list(hist, encoded = True):
    timesteps = get_timesteps(hist)
    labels = []
    for timestep in timesteps:
        labels.append(get_label(timestep, encoded))
    return np.array(labels)
labels_list_example = get_labels_list(opponent_hist + player_hist)
labels_list_example.shape

(80,)

In [149]:
hist = opponent_hist + player_hist
features = get_features_list(hist)
labels = get_labels_list(hist)

features_onehot = tf.one_hot(features, len(vocab))
labels_onehot = tf.one_hot(labels, len(vocab_moves))

features_onehot.shape, labels_onehot.shape

(TensorShape([80, 58, 363]), TensorShape([80, 3]))

In [ ]:
model = tf.keras.Sequential([
    layers.LSTM(1024, input_shape=(features_onehot.shape[1], features_onehot.shape[2])),
    layers.Dense(1024, activation='relu'),
    layers.Dense(labels_onehot.shape[1], activation='softmax')
])

model.compile(loss='CategoricalCrossentropy',
            optimizer='adam',
            metrics=['Accuracy'])

history = model.fit(
    features_onehot,
    labels_onehot,
    epochs=100)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 739ms/step - Accuracy: 0.3508 - loss: 1.1036
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 765ms/step - Accuracy: 0.6617 - loss: 1.3772
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 736ms/step - Accuracy: 0.7602 - loss: 0.9650
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 733ms/step - Accuracy: 0.7273 - loss: 0.9153
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 774ms/step - Accuracy: 0.4781 - loss: 0.8446
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 748ms/step - Accuracy: 0.8305 - loss: 0.6201
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 734ms/step - Accuracy: 0.9734 - loss: 0.4723
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 760ms/step - Accuracy: 0.5367 - loss: 2.6590
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 736ms/step - Accuracy: 0.8031 - loss: 0.6479
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 750ms/step - Accuracy: 0.7742 - loss: 0.5601
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 742ms/step - Accuracy: 0.8562 - loss: 0.4820
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 747ms/step - Accuracy:

In [152]:
def predict(opponent, player):
    # input: opponent and player histories
    # output: predicted output and it's probability
    m = min(len(opponent), len(player)) #if the inputs have different lengths, 'm' is the smaller length,
    seq = opponent[-m :] + player[-m :] #only use the last 'm' plays of each player
    print(seq)
    features = get_features(seq) #get features for the last 'm' plays
    print(features)
    features_onehot = tf.one_hot([features], len(vocab)) #one-hot encode the features

    pred = model.predict(features_onehot) #run the prediction. Return list of probabilities for next char
    print(pred)
    idx = np.argmax(pred) #idx of the char with max probability, equal to the integer encoding for the chars
    return int_to_text([idx]), round(float(pred[0][idx]), 2) #predicted char, probability

predict('RRRPPPSSS'*10, 'PPPPPPPPP'*10)

RRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP
[2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 0, 0, 11, 38, 65, 146, 3, 12, 39, 120, 11, 3, 14, 25, 47, 47, 146, 146, 3, 3, 12, 12, 39, 39, 120, 120, 3, 25, 47, 146, 3, 12, 39, 120]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
[[1.5839889e-06 9.9999797e-01 4.7292127e-07]]


('R', 1.0)